In [2]:
import torch

- https://iq.opengenus.org/as-strided-op-in-pytorch/
- https://medium.com/@heyamit10/best-way-to-cut-a-pytorch-tensor-into-overlapping-chunks-14d80a99919c

## What does as_strided do?  
as_strided는 원하는 모양의 텐서를 뽑아낼 때 사용됩니다.<br>
그 문법은 다음과 같습니다: <br>

``` python
as_strided(input, size, stride, storage_offset)
```
Where: <br>
- input = Is an input tensor
- size = The shape of the output tensor you wish to make (Specified by Tuple of Ints / Int)
- stride = The types of steps you make while creating your output tensor (Specified by Tuple of Ints / Int)
- storage_offset = Start position- Number of tensors as an offset you want to use before starting the creation.



## Basic Example  
Say I have 3x3 matrix A  

$ A = \begin{bmatrix} 1 & 2 & 3 \\ 4 & 5 & 6\end{bmatrix} $

I wish to create a 2 $\times$ 2 matrix B which is a subset of A  

$ B = \begin{bmatrix} 1 & 2 \\ 4 & 5\end{bmatrix} $  

To do this, we see that all we need to do is to move between rows of our matrix I.e.

- Go to element 1, go to the next row to retrieve 4
- Go to element 2, go to the next row to retrieve 5

as_strided는 행렬을 긴 리스트로 해석하기 때문에, 기본적으로는 A를 다음과 같은 표현으로 바라보는 것과 같습니다:

In [ ]:
# As strided looks at these matrices as:
A = [1, 2, 3, 4, 5, 6]

즉, 행(row) 사이를 이동하려면 stride를 행렬의 한 행의 길이와 같게 설정해야 한다는 뜻입니다. 이 경우에는 3이 됩니다. <br>
이를 as_strided로 구현하면 다음과 같습니다: <br>

In [ ]:
A = torch.tensor([[1, 2, 3], [4, 5, 6]])
B = torch.as_strided(A, (2,2), (1, 3))

## Breaking Down the stride tuple:  
이 함수 호출에서 stride 튜플이 어떻게 동작하는지 궁금할 수 있는데, 여기 그 해설이   
있습니다.  

- stride 연산은 원소를 가져오는 과정으로 생각할 수 있습니다. 즉, 하나의 창(window)이 계속해서 움직이며 슬라이딩하는 방식입니다. 이전 예제에서 그 창은 단일 원소-즉, 원소 1을 가리키고 있었습니다.  
- stride 튜플의 첫 번째 원소는 그 창이 얼마나 움직여야 하는지를 지정합니다. 이 경우, 우리는 수평으로 한 칸씩 움직이고 싶습니다. **이를 저는 `i`라고 부르겠습니다.**
- stride 튜플의 두 번째 원소는 원소를 가져오기 전에 몇 단계 건너뛸지를 지정합니다. 예를 들어, 우리가 현재 원소 `1`에 있을 때, 이제 `i+3`번째 인덱스를 가져오고 싶은 것입니다. <br>
<br>
이후에는 이보다 더 복잡한 예제를 보게 되겠지만, 이제 기본 개념을 이해했으니 좀 더 실용적인 예제로 들어갈 수 있습니다.  



## A more practical example of as_strided:
Say for example, we wish to perform a matrix trace on a 3x3 matrix I.e.  
<br>
$ X = \begin{bmatrix} 1 & 2 & 3 \\ 4 & 5 & 6\\ 7 & 8 & 9 \end{bmatrix} $


Our tensor representation of this would be:

In [6]:
X = torch.tensor([[1, 2, 3], [4,5,6], [7,8,9]])
print(X)

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])


Mathematically, this can be represented as:   
$$ \sum_{i=1}^3 a_{ii} = a_{11}+a_{22}+a_{33} $$  
This can be implemented using some form of for loop:  

In [10]:
x_size = len(X[0])
trace = 0		
for x in range(3):
	idx = x * x_size + x
	trace += X.flatten()[idx]
print(trace)

tensor(15)


To convert this into an **as_strided form**, we can differentiate the equation for the index. I.e.  
If our for loop equation is:  

$ index = x \times \lvert x \rvert + x $  

Differentiating with respect to x gives us:  
$$\frac{\partial index}{\partial x}=\lvert x \rvert + 1$$



We can then sub this in for our as_strided implementation  

In [13]:
n = X.clone()
x_size = len(X[0])
trace = torch.as_strided(n, (x_size,), (x_size + 1,)).sum()
print(trace)

tensor(15)


In [16]:
tensor.size(0)

10

In [14]:
# Example 1D tensor
tensor = torch.arange(10)
chunk_size = 3
overlap = 1

# Calculate stride based on the overlap
stride = chunk_size - overlap

# Use as_strided to create overlapping chunks
chunks = tensor.as_strided(size=(tensor.size(0) - chunk_size + 1, chunk_size),
                           stride=(stride, 1))

print("Original Tensor:", tensor)
print("Overlapping Chunks:\n", chunks)

RuntimeError: setStorage: sizes [8, 3], strides [2, 1], storage offset 0, and itemsize 8 requiring a storage size of 136 are out of bounds for storage of size 80

In [21]:
tensor = torch.arange(10)
chunk_size = 3
overlap = 1

# 이동 간격(step) = chunk_size - overlap
step = chunk_size - overlap
if step <= 0:
    raise ValueError("overlap must be < chunk_size and non-negative")

N = tensor.numel()
if N < chunk_size:
    raise ValueError("tensor length must be >= chunk_size")

# 청크 개수 계산 (범위 초과 방지)
num_chunks = 1 + (N - chunk_size) // step

# as_strided로 겹치는 청크 만들기
# 바깥(청크) 차원의 stride는 'step', 안쪽(청크 내부)은 1
chunks = tensor.as_strided(size=(num_chunks, chunk_size),
                           stride=(step, 1))

print("Original Tensor:", tensor)
print("Overlapping Chunks:\n", chunks)

Original Tensor: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
Overlapping Chunks:
 tensor([[0, 1, 2],
        [2, 3, 4],
        [4, 5, 6],
        [6, 7, 8]])


In [18]:
num_chunks

4

In [19]:
step

2

In [26]:
X

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])

In [27]:
patches = X.as_strided(size=(2, 2, 2), stride=(1, 3, 1))
print(patches)

tensor([[[1, 2],
         [4, 5]],

        [[2, 3],
         [5, 6]]])


In [29]:
patches = X.as_strided(size=(2,2, 2, 2), stride=(2, 1, 3, 1))
print(patches)

tensor([[[[1, 2],
          [4, 5]],

         [[2, 3],
          [5, 6]]],


        [[[3, 4],
          [6, 7]],

         [[4, 5],
          [7, 8]]]])


In [30]:
x = torch.arange(1, 17).view(4, 4)

In [31]:
x

tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12],
        [13, 14, 15, 16]])

In [32]:
# 원본 5x5 행렬
x = torch.arange(1, 26).view(5, 5)
print("원본 행렬:\n", x)

원본 행렬:
 tensor([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10],
        [11, 12, 13, 14, 15],
        [16, 17, 18, 19, 20],
        [21, 22, 23, 24, 25]])


In [ ]:
[[[1,2],[6,7],[[2,3],[7,8]],[[3,4],[8,9],[[4,5],[9,10]]]],
 []   
]